# IBM Applied Data Science Capstone Course by Coursera

### Week 5 Final Report

#### Opening a New dessert bar in San Diego, California

-    Build a dataframe of neighborhoods in Kuala Lumpur, Malaysia by web scraping the data from Wikipedia page
-    Get the geographical coordinates of the neighborhoods
-    Obtain the venue data for the neighborhoods from Foursquare API
-    Explore and cluster the neighborhoods
-    Select the best cluster to open a new shopping mall

#### 1. Import libraries

In [176]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.



#### 2. Scrap data from Wikipedia page into a DataFrame


In [177]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_San_Diego").text

In [178]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [179]:
# create a list to store neighborhood data
neighborhoodList = []

In [180]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [181]:
# create a new DataFrame from the list
sd_df = pd.DataFrame({"Neighborhood": neighborhoodList})

sd_df = sd_df.drop(0)
sd_df = sd_df.drop(1)
sd_df = sd_df.drop(2)


In [182]:
sd_df

,Neighborhood
3,"Allied Gardens, San Diego"
4,"Alta Vista, San Diego"
5,"Alvarado Estates, San Diego"
6,"Azalea Park, San Diego"
7,"Bankers Hill, San Diego"
8,"Barrio Logan, San Diego"
9,"Bay Terraces, San Diego"
10,"Bird Rock, San Diego"
11,"Birdland, San Diego"
12,"Black Mountain Ranch, San Diego"


In [183]:
sd_df.shape

(127, 1)

#### 3. Get the geographical coordinates

In [184]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, San Diego, CA'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [185]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in sd_df["Neighborhood"].tolist() ]

In [186]:
coords

[[32.79631000000006, -117.09452999999996],
 [32.693160000000034, -117.06778999999995],
 [32.77743098760981, -117.062271],
 [32.73284000000007, -117.10775999999998],
 [32.728480025535305, -117.16124004053883],
 [32.69467000000003, -117.13807999999995],
 [32.69166000000007, -117.04083999999995],
 [32.813120000000026, -117.26510999999999],
 [32.79290000000003, -117.15430999999995],
 [32.99950000000007, -117.14514999999994],
 [32.73105000000004, -117.05379999999997],
 [32.733755746072305, -117.1273439057351],
 [32.97878990705475, -117.08510654814945],
 [32.715050000000076, -117.09319999999997],
 [32.749690000000044, -117.10714999999999],
 [32.7773667771192, -117.14775896124173],
 [32.83456000000007, -117.19440999999995],
 [32.74150139458533, -117.05027939458533],
 [32.72229503553981, -117.16740899999999],
 [32.71568000000008, -117.16170999999997],
 [32.718920000000026, -117.15652999999998],
 [32.78762000000006, -117.06416999999999],
 [32.948390000000074, -117.25991999999997],
 [32.94257000

In [187]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [188]:
# merge the coordinates into the original dataframe
sd_df['Latitude'] = df_coords['Latitude']
sd_df['Longitude'] = df_coords['Longitude']

In [189]:
# check the neighborhoods and the coordinates
print(sd_df.shape)
sd_df

(127, 3)


,Neighborhood,Latitude,Longitude
3,"Allied Gardens, San Diego",32.732840,-117.107760
4,"Alta Vista, San Diego",32.728480,-117.161240
5,"Alvarado Estates, San Diego",32.694670,-117.138080
6,"Azalea Park, San Diego",32.691660,-117.040840
7,"Bankers Hill, San Diego",32.813120,-117.265110
8,"Barrio Logan, San Diego",32.792900,-117.154310
9,"Bay Terraces, San Diego",32.999500,-117.145150
10,"Bird Rock, San Diego",32.731050,-117.053800
11,"Birdland, San Diego",32.733756,-117.127344
12,"Black Mountain Ranch, San Diego",32.978790,-117.085107


In [190]:
sd_df = sd_df.drop(127)
sd_df = sd_df.drop(128)
sd_df = sd_df.drop(129)

In [191]:
# save the DataFrame as CSV file
sd_df.to_csv("sd_df.csv", index=False)

In [192]:
sd_df

,Neighborhood,Latitude,Longitude
3,"Allied Gardens, San Diego",32.732840,-117.107760
4,"Alta Vista, San Diego",32.728480,-117.161240
5,"Alvarado Estates, San Diego",32.694670,-117.138080
6,"Azalea Park, San Diego",32.691660,-117.040840
7,"Bankers Hill, San Diego",32.813120,-117.265110
8,"Barrio Logan, San Diego",32.792900,-117.154310
9,"Bay Terraces, San Diego",32.999500,-117.145150
10,"Bird Rock, San Diego",32.731050,-117.053800
11,"Birdland, San Diego",32.733756,-117.127344
12,"Black Mountain Ranch, San Diego",32.978790,-117.085107


#### 4. Create a map of San Diego with neighborhoods superimposed on top

In [193]:
# get the coordinates for San Diego
address = 'San Diego, CA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate for San Diego, CA is {}, {}.'.format(latitude, longitude))

The geograpical coordinate for San Diego, CA is 32.7174209, -117.1627714.


In [194]:
# create map of San Diego using latitude and longitude values
map_sd = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(sd_df['Latitude'], sd_df['Longitude'], sd_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sd)  
    
map_sd

In [216]:
# save the map as HTML file
map_sd.save('map_sd.html')

#### 5. Use the Foursquare API to explore the neighborhoods

In [217]:
from config import client_id, client_secret

In [218]:
# define Foursquare Credentials and Version
CLIENT_ID = client_id # your Foursquare ID
CLIENT_SECRET = client_secret # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails loaded successfully')


Your credentails loaded successfully


#### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [219]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sd_df['Latitude'], sd_df['Longitude'], sd_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [220]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(9767, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Allied Gardens, San Diego",32.73284,-117.10776,City Farmers Nursery,32.737851,-117.092571,Garden Center
1,"Allied Gardens, San Diego",32.73284,-117.10776,Nate's Garden Grill,32.737970,-117.092530,American Restaurant
2,"Allied Gardens, San Diego",32.73284,-117.10776,Thorn Street Brewery,32.739407,-117.125496,Brewery
3,"Allied Gardens, San Diego",32.73284,-117.10776,Chiquita's Mexican Restaurant,32.723732,-117.107230,Mexican Restaurant
4,"Allied Gardens, San Diego",32.73284,-117.10776,Azalea Community Park,32.732355,-117.106325,Park




#### Let's check how many venues were returned for each neighorhood


In [221]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Allied Gardens, San Diego",69,69,69,69,69,69
"Alta Vista, San Diego",100,100,100,100,100,100
"Alvarado Estates, San Diego",100,100,100,100,100,100
"Azalea Park, San Diego",38,38,38,38,38,38
"Bankers Hill, San Diego",83,83,83,83,83,83
"Barrio Logan, San Diego",69,69,69,69,69,69
"Bay Terraces, San Diego",13,13,13,13,13,13
"Bird Rock, San Diego",73,73,73,73,73,73
"Birdland, San Diego",100,100,100,100,100,100


####  Let's find out how many unique categories can be curated from all the returned venues


In [222]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 362 uniques categories.


In [223]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Garden Center', 'American Restaurant', 'Brewery',
       'Mexican Restaurant', 'Park', 'Pet Store', 'Grocery Store',
       'Dive Bar', 'Juice Bar', 'Coffee Shop',
       'Latin American Restaurant', 'Restaurant', 'Asian Restaurant',
       'Wings Joint', 'Sandwich Place', 'Taco Place',
       'Thrift / Vintage Store', 'Convenience Store', 'Snack Place',
       'Pharmacy', 'Video Game Store', 'Vietnamese Restaurant',
       'Chinese Restaurant', 'Bank', 'Liquor Store', 'Discount Store',
       'Fast Food Restaurant', 'Pizza Place', 'Diner', 'Shipping Store',
       'Intersection', 'Nature Preserve', 'New American Restaurant',
       'Trail', 'Seafood Restaurant', 'Ethiopian Restaurant',
       'Bus Station', 'Food Truck', 'Italian Restaurant', 'Bakery',
       'Thai Restaurant', 'Food', 'Sushi Restaurant', 'Deli / Bodega',
       'South American Restaurant', 'Bar', 'Spa', 'Ramen Restaurant',
       'Soccer Field', 'Steakhouse'], dtype=object)

In [224]:
# check if the results contain "dessert bar"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

###  6. Analyze Each Neighborhood

In [228]:
# one hot encoding
sd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sd_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sd_onehot.columns[-1]] + list(sd_onehot.columns[:-1])
sd_onehot = sd_onehot[fixed_columns]

print(sd_onehot.shape)
sd_onehot.head()

(9767, 363)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Alternative Healer,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Cafeteria,College Football Field,College Rec Center,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Bathroom,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Recruiting Agency,Rental Car Location,Reservoir,Residential Building (

####  Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each categor


In [229]:
sd_grouped = sd_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sd_grouped.shape)
sd_grouped

(124, 363)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Alternative Healer,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Cafeteria,College Football Field,College Rec Center,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Bathroom,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Recruiting Agency,Rental Car Location,Reservoir,Residential Building (

In [230]:
len(sd_grouped[sd_grouped["Chinese Restaurant"] > 0])

58

#### Create a new DataFrame for Shopping Mall data only


In [231]:
sd_chinese = sd_grouped[["Neighborhoods","Chinese Restaurant"]]
sd_chinese

,Neighborhoods,Chinese Restaurant
0,"Allied Gardens, San Diego",0.028986
1,"Alta Vista, San Diego",0.000000
2,"Alvarado Estates, San Diego",0.020000
3,"Azalea Park, San Diego",0.026316
4,"Bankers Hill, San Diego",0.000000
5,"Barrio Logan, San Diego",0.000000
6,"Bay Terraces, San Diego",0.000000
7,"Bird Rock, San Diego",0.013699
8,"Birdland, San Diego",0.000000
9,"Black Mountain Ranch, San Diego",0.040000


#### 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [232]:
# set number of clusters
kclusters = 3

sd_clustering = sd_chinese.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 1, 2, 2, 2, 1, 2, 0])

In [233]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sd_merged = sd_chinese.copy()


# add clustering labels
sd_merged["Cluster Labels"] = kmeans.labels_

In [234]:
sd_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sd_merged.head()

,Neighborhood,Chinese Restaurant,Cluster Labels
0,"Allied Gardens, San Diego",0.028986,0
1,"Alta Vista, San Diego",0.000000,2
2,"Alvarado Estates, San Diego",0.020000,1
3,"Azalea Park, San Diego",0.026316,1
4,"Bankers Hill, San Diego",0.000000,2


In [235]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sd_merged = sd_merged.join(sd_df.set_index("Neighborhood"), on="Neighborhood")

print(sd_merged.shape)
sd_merged.head() # check the last columns!

(124, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,"Allied Gardens, San Diego",0.028986,0,32.73284,-117.10776
1,"Alta Vista, San Diego",0.000000,2,32.72848,-117.16124
2,"Alvarado Estates, San Diego",0.020000,1,32.69467,-117.13808
3,"Azalea Park, San Diego",0.026316,1,32.69166,-117.04084
4,"Bankers Hill, San Diego",0.000000,2,32.81312,-117.26511


In [237]:
# sort the results by Cluster Labels
print(sd_merged.shape)
sd_merged.sort_values(["Cluster Labels"], inplace=True)
sd_merged.head()

(124, 5)


,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,"Allied Gardens, San Diego",0.028986,0,32.73284,-117.107760
40,"Harbor Island, San Diego",0.050000,0,32.74031,-117.091100
86,"Point Loma Heights, San Diego",0.066667,0,32.92718,-117.019600
42,"Hillcrest, San Diego",0.050000,0,32.82181,-117.149620
9,"Black Mountain Ranch, San Diego",0.040000,0,32.97879,-117.085107


#### Finally, let's visualize the resulting clusters


In [238]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sd_merged['Latitude'], sd_merged['Longitude'], sd_merged['Neighborhood'], sd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [240]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### 8. Examine Clusters

##### Cluster 0

In [243]:
sd_merged.loc[sd_merged['Cluster Labels'] == 0]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
0,"Allied Gardens, San Diego",0.028986,0,32.732840,-117.107760
40,"Harbor Island, San Diego",0.050000,0,32.740310,-117.091100
86,"Point Loma Heights, San Diego",0.066667,0,32.927180,-117.019600
42,"Hillcrest, San Diego",0.050000,0,32.821810,-117.149620
9,"Black Mountain Ranch, San Diego",0.040000,0,32.978790,-117.085107
114,"Talmadge, San Diego",0.029412,0,32.966860,-117.154200
43,"Islenair, San Diego",0.031250,0,32.764940,-117.105010
59,"Middletown, San Diego",0.035088,0,32.912250,-117.150720
111,"Stockton, San Diego",0.050000,0,32.762400,-117.089440
106,"South Park, San Diego",0.042105,0,32.678862,-117.109876


##### Cluster 1

In [244]:
sd_merged.loc[sd_merged['Cluster Labels'] == 1]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
48,"La Jolla Village, San Diego",0.010000,1,32.784190,-117.171350
85,"Paradise Hills, San Diego",0.011364,1,33.024440,-117.084830
92,"Rolando Park, San Diego",0.015625,1,32.944730,-117.092320
90,"Rancho Peñasquitos, San Diego",0.010000,1,32.761110,-117.059290
91,"Redwood Village, San Diego",0.010000,1,32.726300,-117.237690
123,"Valencia Park, San Diego",0.014706,1,32.714520,-117.244550
55,"Lomita, San Diego",0.020000,1,32.695980,-117.125630
46,"Kensington, San Diego",0.011628,1,32.718300,-117.233700
39,"Grantville, San Diego",0.020000,1,32.749950,-117.165560
35,"Gaslamp Quarter, San Diego",0.010000,1,32.711260,-117.133850


##### Cluster 2

In [245]:
sd_merged.loc[sd_merged['Cluster Labels'] == 2]

,Neighborhood,Chinese Restaurant,Cluster Labels,Latitude,Longitude
65,"Mission Hills, San Diego",0.0,2,32.711480,-117.108930
45,"Kearny Mesa, San Diego",0.0,2,32.861620,-117.233220
49,"La Playa, San Diego",0.0,2,32.725860,-117.167600
50,"Lincoln Park, San Diego",0.0,2,32.711550,-117.144210
51,"Linda Vista, San Diego",0.0,2,32.740200,-117.238660
52,"Little Italy, San Diego",0.0,2,32.763906,-117.144327
56,"Marina, San Diego",0.0,2,32.737010,-117.175750
54,"Loma Portal, San Diego",0.0,2,32.727055,-117.124253
64,"Mission Beach, San Diego",0.0,2,32.769490,-117.190070
57,"Marston Hills, San Diego",0.0,2,32.715680,-117.161710


#### Observations:

Most of the Chinese Restaurants are concentrated in the central area of San Diego, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no Chinese restaurants in the neighborhoods. This represents a great opportunity and high potential areas to open new Chinese restaurants as there is very little to no competition from existing Chinese restaurants. Meanwhile, Chinese restaurants in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of Chinese restaurant. From another perspective, this also shows that the oversupply of Chinese restaurants mostly happened in the central area of San Diego, with the suburbs only have a few restaurants. Therefore, this research recommends that restaurant owners capitalize on these findings to open new Chinese restaurants in neighborhoods in cluster 0 with little to no competition. Restaurant owners with unique selling points that stand out from the competition can also open new Chinese restaurants in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of Chinese restaurant and suffering from intense competition.

